In [1]:
%run data.py

In [2]:
zipcode_data = fetchData(nyc_zcta_url)
zipcode_data.head(10)

,MODZCTA,Positive,Total,zcta_cum.perc_pos
0,NaN,1626,1941,83.77
1,10001.0,318,1090,29.17
2,10002.0,924,2401,38.48
3,10003.0,413,1504,27.46
4,10004.0,28,101,27.72
5,10005.0,54,250,21.60
6,10006.0,23,106,21.70
7,10007.0,48,238,20.17
8,10009.0,631,2062,30.60
9,10010.0,241,1137,21.20


In [3]:
zipcode_old_df = spark.createDataFrame(zipcode_data)

zipcode_df = zipcode_old_df.select(zipcode_old_df.MODZCTA.cast("integer").alias("zipcode"),
                                zipcode_old_df["Positive"], zipcode_old_df["Total"])

zipcode_df = zipcode_df.filter(zipcode_df["zipcode"] != 0)

In [4]:
results_df = fetch_nycOpenData(nyc_income, 100, 10000)
results_df.head(10)


,project_id,project_name,program_group,project_start_date,building_id,house_number,street_name,borough,postcode,bbl,...,_3_br_units,_4_br_units,_5_br_units,_6_br_units,unknown_br_units,counted_rental_units,counted_homeownership_units,all_counted_units,total_units,project_completion_date
0,64390,COOPER SQUARE MHA - PHASE 1.GHPP.FY20,Multifamily Finance Program,2020-03-31T00:00:00.000,1564,83,2 AVENUE,Manhattan,10003,1004600035,...,4,0,0,0,0,1,3,4,4,NaN
1,64390,COOPER SQUARE MHA - PHASE 1.GHPP.FY20,Multifamily Finance Program,2020-03-31T00:00:00.000,10537,13,EAST 3 STREET,Manhattan,10003,1004590043,...,0,0,0,0,0,1,9,10,10,NaN
2,64390,COOPER SQUARE MHA - PHASE 1.GHPP.FY20,Multifamily Finance Program,2020-03-31T00:00:00.000,10557,21,EAST 3 STREET,Manhattan,10003,1004590039,...,0,0,0,0,0,5,11,16,16,NaN
3,64390,COOPER SQUARE MHA - PHASE 1.GHPP.FY20,Multifamily Finance Program,2020-03-31T00:00:00.000,10569,23,EAST 3 STREET,Manhattan,10003,1004590038,...,0,0,0,0,0,3,14,17,17,NaN
4,64390,COOPER SQUARE MHA - PHASE 1.GHPP.FY20,Multifamily Finance Program,2020-03-31T00:00:00.000,10580,25,EAST 3 STREET,Manhattan,10003,1004590037,...,0,0,0,0,0,2,16,18,18,NaN
5,64390,COOPER SQUARE MHA - PHASE 1.GHPP.FY20,Multifamily Finance Program,2020-03-31T00:00:00.000,10582,27,EAST 3 STREET,Manhattan,10003,1004590036,...,0,0,0,0,0,1,12,13,13,NaN
6,64390,COOPER SQUARE MHA - PHASE 1.GHPP.FY20,Multifamily Finance Program,2020-03-31T00:00:00.000,10659,89,EAST 3 STREET,Manhattan,10003,1004450042,...,0,0,0,0,0,0,15,15,15,NaN
7,64390,COOPER SQUARE MHA - PHASE 1.GHPP.FY20,Multifamily Finance Program,2020-03-31T00:00:00.000,10660,9,EAST 3 STREET,Manhattan,10003,1004590045,...,2,0,0,0,0,0,13,13,13,NaN
8,64390,COOPER SQUARE MHA - PHASE 1.GHPP.FY20,Multifamily Finance Program,2020-03-31T00:00:00.000,10755,56,EAST 4 STREET,Manhattan,10003,1004590014,...,0,0,0,0,0,2,17,19,19,NaN
9,64390,COOPER SQUARE MHA - PHASE 1.GHPP.FY20,Multifamily Finance Program,2020-03-31T00:00:00.000,10756,57,EAST 4 STREET,Manhattan,10003,1004600059,...,0,0,0,0,0,2,15,17,17,NaN


In [5]:
df = pandas_to_spark(results_df)
df = df.filter(df["postcode"].isNotNull())
df = df.filter(df["postcode"] != "NaN")

In [6]:
new_df = df.select(df["building_id"], df["postcode"], 
                           df.extremely_low_income_units.cast('integer').alias('extremely_low_income_units'),
                           df.very_low_income_units.cast('integer').alias('very_low_income_units'),
                           df.low_income_units.cast('integer').alias('low_income_units'),
                           df.middle_income_units.cast('integer').alias('middle_income_units'),
                           df.moderate_income_units.cast('integer').alias('moderate_income_units'),
                           df.other_income_units.cast('integer').alias('other_income_units')
                          )
new_df.show()
new_df.createOrReplaceTempView("new_df")

+-----------+--------+--------------------------+---------------------+----------------+-------------------+---------------------+------------------+
|building_id|postcode|extremely_low_income_units|very_low_income_units|low_income_units|middle_income_units|moderate_income_units|other_income_units|
+-----------+--------+--------------------------+---------------------+----------------+-------------------+---------------------+------------------+
|       1564|   10003|                         0|                    4|               0|                  0|                    0|                 0|
|      10537|   10003|                         0|                   10|               0|                  0|                    0|                 0|
|      10557|   10003|                        15|                    1|               0|                  0|                    0|                 0|
|      10569|   10003|                        14|                    3|               0|            

In [8]:
income_old_df = spark.sql("SELECT postcode as zipcode, \
                                 sum(extremely_low_income_units) as extremely_low_income_units, \
                                 sum(very_low_income_units) as very_low_income_units, \
                                 sum(low_income_units) as low_income_units, \
                                 sum(middle_income_units) as middle_income_units, \
                                 sum(moderate_income_units) as moderate_income_units, \
                                 sum(other_income_units) as other_income_units \
                                 FROM new_df GROUP BY postcode ORDER BY postcode")

income_old_df.createOrReplaceTempView("income_old_df")
income_df = spark.sql("SELECT zipcode, \
                                (extremely_low_income_units + very_low_income_units + low_income_units) as low_income_units, \
                                middle_income_units, \
                                moderate_income_units \
                                FROM income_old_df")

income_df.show()

+-------+----------------+-------------------+---------------------+
|zipcode|low_income_units|middle_income_units|moderate_income_units|
+-------+----------------+-------------------+---------------------+
|  10001|            1581|                305|                  410|
|  10002|            3882|                199|                  566|
|  10003|             575|                  0|                    0|
|  10009|            1500|               4013|                   79|
|  10010|             577|               1149|                  116|
|  10011|             482|                237|                  187|
|  10012|             175|                  0|                    0|
|  10013|              88|                  0|                    0|
|  10014|              45|                  7|                    0|
|  10016|             619|                  7|                    0|
|  10017|              87|                 22|                    0|
|  10018|             216|        

In [17]:
income_df = income_df.withColumn("income", 
                                                  f.when((f.col("low_income_units") >=  f.col("middle_income_units")) & \
                                                             (f.col("low_income_units") >=  f.col("moderate_income_units")), "low")\
                                                  .when((f.col("middle_income_units") >=  f.col("low_income_units")) & \
                                                            (f.col("middle_income_units") >=  f.col("moderate_income_units")), "middle")\
                                                  .when((f.col("moderate_income_units") >=  f.col("low_income_units")) & \
                                                            (f.col("moderate_income_units") >=  f.col("middle_income_units")), "moderate"))

In [18]:
income_df.show()

+-------+----------------+-------------------+---------------------+------+
|zipcode|low_income_units|middle_income_units|moderate_income_units|income|
+-------+----------------+-------------------+---------------------+------+
|  10001|            1581|                305|                  410|   low|
|  10002|            3882|                199|                  566|   low|
|  10003|             575|                  0|                    0|   low|
|  10009|            1500|               4013|                   79|middle|
|  10010|             577|               1149|                  116|middle|
|  10011|             482|                237|                  187|   low|
|  10012|             175|                  0|                    0|   low|
|  10013|              88|                  0|                    0|   low|
|  10014|              45|                  7|                    0|   low|
|  10016|             619|                  7|                    0|   low|
|  10017|   

In [20]:
zipcode_df.createOrReplaceTempView("zipcode_df")
income_df.createOrReplaceTempView("income_df")

join_df = spark.sql("SELECT * FROM zipcode_df NATURAL JOIN income_df")
join_df.createOrReplaceTempView("join_df")

final_df = spark.sql("SELECT income, sum(Positive) as positive_num FROM join_df GROUP BY income ORDER BY positive_num")
data = final_df.toPandas()
data.head(10)

,income,positive_num
0,moderate,4187
1,middle,11812
2,low,128535


In [23]:
import seaborn as sns
cm = sns.light_palette("#ff7000", as_cmap = True)
data.style.background_gradient(cmap = cm)

,income,positive_num
0,moderate,4187
1,middle,11812
2,low,128535
